In [ ]:
!pip install --upgrade Pillow
!pip install tensorflow==1.11
!pip install ipdb
import ipdb
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim 
from torchvision import datasets
from torch.autograd import Variable
import numpy as np
import pandas as pd
import cv2
import argparse
import os
from PIL import Image, ImageEnhance, ImageOps
from tqdm import tqdm
import random

# Training settings
class Args:
  load_dir = 'drive/My Drive/Bird_Classification/features'
  data = "drive/My Drive/Bird_Classification/data"
  img_size=299
  batch_size=32
  epochs=50
  lr=0.01
  momentum=0.9
  weight_decay=3e-4
  grad_clip=5.
  seed=1
  log_interval=10
  experiment='drive/My Drive/Bird_Classification/experiment'
  outfile='drive/My Drive/experiment/kaggle.csv'
  pth='drive/My Drive/Bird_Classification/experiment/InceptionV3_2_Layers/model_21_95.pth'

args = Args()
use_cuda = torch.cuda.is_available()
torch.manual_seed(args.seed)

# Get files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Create experiment folder
if not os.path.isdir(args.experiment):
    os.makedirs(args.experiment)

# Inception3 model

In [0]:
# Feature extraction on original dataset with pre-trained on ImageNet and iNat2017 Inception3
!python "drive/My Drive/Bird_Classification/cvpr18-inaturalist-transfer-master/feature_extraction.py" --dataset "bird_dataset" --all

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [0]:
# Feature extraction on cropped dataset with pre-trained on ImageNet and iNat2017 Inception3
!python "drive/My Drive/Bird_Classification/cvpr18-inaturalist-transfer-master/feature_extraction.py" --dataset "crop_dataset" --all

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [0]:
# Additionnal test features extraction for test time augmentation
additionnal_test_sets = ['flip_bird_dataset', 'flip_crop_bird_dataset']
for test_set in additionnal_test_sets:
  !python "drive/My Drive/Bird_Classification/cvpr18-inaturalist-transfer-master/feature_extraction.py" --dataset test_set

In [3]:
# 2-layers network of the features

# Features and labels
features_train = np.load(os.path.join(args.load_dir, 'bird_dataset_dense_feature_train.npy'))
labels_train = np.load(os.path.join(args.load_dir, 'bird_dataset_dense_label_train.npy'))
features_val = np.load(os.path.join(args.load_dir, 'bird_dataset_dense_feature_val.npy'))
labels_val = np.load(os.path.join(args.load_dir, 'bird_dataset_dense_label_val.npy'))

features_train_crop = np.load(os.path.join(args.load_dir, 'augmented_dataset_dense_feature_train.npy'))
features_val_crop = np.load(os.path.join(args.load_dir, 'augmented_dataset_dense_feature_val.npy'))

features_train = np.concatenate((features_train, features_train_crop), axis=1)
features_val = np.concatenate((features_val, features_val_crop), axis=1)

# Dataloaders
features_tensor = torch.stack([torch.Tensor(i) for i in features_train]) 
labels_tensor = torch.stack([torch.Tensor([i]) for i in labels_train])
train_data = torch.utils.data.TensorDataset(features_tensor, labels_tensor) 
features_tensor = torch.stack([torch.Tensor(i) for i in features_val])
labels_tensor = torch.stack([torch.Tensor([i]) for i in labels_val])
val_data = torch.utils.data.TensorDataset(features_tensor,labels_tensor) 
train_loader = torch.utils.data.DataLoader(train_data, batch_size=args.batch_size, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=args.batch_size, shuffle=False, num_workers=4)

# Model
class Classifier(nn.Module):
    def __init__(self,embedding_dim):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(embedding_dim, 512)
        self.fc2 = nn.Linear(512, 20)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Classifier(features_train.shape[1])

if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

# Optimizer, LR, and criterion
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum, weight_decay=args.weight_decay)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs)
criterion = torch.nn.CrossEntropyLoss()

# Training functions
def train_classifier(model, train_loader, optimizer, lr_scheduler, criterion, epoch):
    lr_scheduler.step()
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
              data, target = Variable(data.cuda()), Variable(target.cuda().long())
        else:
              data, target = Variable(data), Variable(target.long())
        optimizer.zero_grad()
        output = model(data)
        target = target.squeeze(1)
        loss = criterion(output, target)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

def validation_classifier(model, criterion, val_loader):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            if use_cuda:
              data, target = Variable(data.cuda()), Variable(target.cuda().long())
            else:
              data, target = Variable(data), Variable(target.long())
            output = model(data)
            target = target.squeeze(1)
            # sum up batch loss
            validation_loss += criterion(output, target).data.item()
            # get the index of the max log-probability
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    return(100. * correct / len(val_loader.dataset))

Using GPU


In [0]:
# Training the classifier 
for epoch in range(1, args.epochs + 1):
    train_classifier(epoch)
    val_acc=validation_classifier()
    if val_acc>=95:
      # Save only when it is good enough
      model_file = args.experiment + '/model_' + str(epoch) + '.pth'
      torch.save(model.state_dict(), model_file)
      print('Saved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file\n')

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 1 [0/1082 (0%)]	Loss: 2.998858
Train Epoch: 1 [320/1082 (29%)]	Loss: 2.741377
Train Epoch: 1 [640/1082 (59%)]	Loss: 2.085440
Train Epoch: 1 [960/1082 (88%)]	Loss: 1.361625

Validation set: Average loss: 0.0352, Accuracy: 92/103 (89%)
Train Epoch: 2 [0/1082 (0%)]	Loss: 0.676885
Train Epoch: 2 [320/1082 (29%)]	Loss: 0.323246
Train Epoch: 2 [640/1082 (59%)]	Loss: 0.266266
Train Epoch: 2 [960/1082 (88%)]	Loss: 0.237956

Validation set: Average loss: 0.0085, Accuracy: 95/103 (92%)
Train Epoch: 3 [0/1082 (0%)]	Loss: 0.090857
Train Epoch: 3 [320/1082 (29%)]	Loss: 0.141260
Train Epoch: 3 [640/1082 (59%)]	Loss: 0.126691
Train Epoch: 3 [960/1082 (88%)]	Loss: 0.230064

Validation set: Average loss: 0.0077, Accuracy: 96/103 (93%)
Train Epoch: 4 [0/1082 (0%)]	Loss: 0.147218
Train Epoch: 4 [320/1082 (29%)]	Loss: 0.175938
Train Epoch: 4 [640/1082 (59%)]	Loss: 0.049514
Train Epoch: 4 [960/1082 (88%)]	Loss: 0.124837

Validation set: Average loss: 0.0068, Accuracy: 95/103 (92%)
Train Epoch:

In [0]:
# Test

# Test features
features_test = np.load(os.path.join(args.load_dir, 'bird_dataset_dense_feature_test.npy'))
features_test_crop = np.load(os.path.join(args.load_dir, 'augmented_dataset_dense_feature_test.npy'))
features_test = np.concatenate((features_test, features_test_crop), axis=1)

# Loading trained model
state_dict = torch.load(args.pth)
model = Classifier(features_test.shape[1])
model.load_state_dict(state_dict)
model.eval()

if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

# Test 
df = pd.read_csv('drive/My Drive/Bird_Classification/template.csv')
test_list=[]
for line in open(os.path.join(args.data+ '/flip_bird_dataset/test.txt'), 'r'):
    test_list.append(line[:-1])
    
for i in range(len(test_list)):
    img_idx = df[df['Id'] == test_list[i][:-4]].index[0]
    data = torch.tensor(features_test[i])
    if use_cuda:
        data = data.cuda()
    output = model(data)
    pred = torch.max(output.data, 0)[1]
    df.Category[img_idx] = pred.cpu().data.numpy()

df.to_csv('drive/My Drive/Bird_Classification/kaggle.csv',index=False)
print("Succesfully wrote kaggle.csv, you can upload this file to the kaggle competition website")

Using GPU


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Succesfully wrote kaggle.csv, you can upload this file to the kaggle competition website


In [6]:
# Test with Test Time Augmentation

# Test features
features_test = np.load(os.path.join(args.load_dir, 'bird_dataset_dense_feature_test.npy'))
features_test_crop = np.load(os.path.join(args.load_dir, 'augmented_dataset_dense_feature_test.npy'))
features_test = np.concatenate((features_test, features_test_crop), axis=1)

# Additional test features
features_test_flip = np.load(os.path.join(args.load_dir, 'flip_bird_dataset_feature_test.npy'))
features_test_crop_flip = np.load(os.path.join(args.load_dir, 'flip_crop_bird_dataset_feature_test.npy'))
features_test_flip = np.concatenate((features_test_flip, features_test_crop_flip), axis=1)

# Loading trained model
state_dict = torch.load(args.pth)
model = Classifier(features_test.shape[1])
model.load_state_dict(state_dict)
model.eval()

if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

# Test 
df = pd.read_csv('drive/My Drive/Bird_Classification/template.csv')
test_list=[]
for line in open(os.path.join(args.data+ '/flip_bird_dataset/test.txt'), 'r'):
    test_list.append(line[:-1])
  
norm = torch.nn.Softmax(0)
for i in range(len(test_list)):
    img_idx = df[df['Id'] == test_list[i][:-4]].index[0]
    data = torch.tensor(features_test[i])
    data_flip = torch.tensor(features_test_flip[i])
    if use_cuda:
        data = data.cuda()
        data_flip = data_flip.cuda()
    output = norm(model(data))
    output_flip = norm(model(data_flip))
    # max TTA
    confidence1, pred1 = torch.max(output.data, 0)
    confidence2, pred2 = torch.max(output_flip.data, 0)
    index = torch.max(torch.stack((confidence1,confidence2)),0)[1].cpu().data.numpy().item(0)
    pred = torch.stack((pred1,pred2))[index]
    # average TTA
    # pred = torch.max(output.data+output_flip.data, 0)[1]
    df.Category[img_idx] = pred.cpu().data.numpy()

df.to_csv('drive/My Drive/Bird_Classification/kaggle.csv',index=False)
print("Succesfully wrote kaggle.csv, you can upload this file to the kaggle competition website")

Using GPU


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Succesfully wrote kaggle.csv, you can upload this file to the kaggle competition website
